In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.autograd import Variable
import operator
import warnings
warnings.filterwarnings('ignore')
import os
from queue import PriorityQueue
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

from sacrebleu import raw_corpus_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#Define constants here
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
BATCH_SIZE = 32
TEST_BATCH_SIZE = 3
words_to_load = 50000
emb_size = 300
wiki_size = 300
CUDA = True
MAX_LENGTH = 100

In [ ]:
datadir = os.getcwd()
datadir

### Load pretrained word embeddings:
Reference: https://fasttext.cc/docs/en/pretrained-vectors.html

@article{bojanowski2017enriching,
  title={Enriching Word Vectors with Subword Information},
  author={Bojanowski, Piotr and Grave, Edouard and Joulin, Armand and Mikolov, Tomas},
  journal={Transactions of the Association for Computational Linguistics},
  volume={5},
  year={2017},
  issn={2307-387X},
  pages={135--146}
}


In [ ]:
#Reference Lab4 HW2
datadir = os.getcwd()
words_to_load = 5000
with open(datadir + '/data/wiki-news-300d-1M.vec') as f:
    loaded_en_embeddings = np.zeros(((words_to_load+4), wiki_size))
    en_word2id = {}
    en_id2words = {}
    
    en_id2words[PAD_TOKEN] = '<PAD>'
    en_id2words[SOS_TOKEN] = '<SOS>'
    en_id2words[EOS_TOKEN] = '<EOS>'
    en_id2words[UNK_TOKEN] = '<UNK>'
    
    en_word2id['<PAD>'] = PAD_TOKEN
    en_word2id['<SOS>'] = SOS_TOKEN
    en_word2id['<EOS>'] = EOS_TOKEN
    en_word2id['<UNK>'] = UNK_TOKEN
    
    en_ordered_words= []
    en_ordered_words.append('<PAD>')
    en_ordered_words.append('<SOS>')
    en_ordered_words.append('<EOS>')
    en_ordered_words.append('<UNK>')
    
    for i, line in enumerate(f):
        if i >= words_to_load:
            break
        if i ==0:#Ignore the first line
            continue;
        s = line.split()
        #print(len(s))
        loaded_en_embeddings[i+4,:] = np.asarray(s[1:])
        en_word2id[s[0]] = i+4 #for extra pad and unk eos and unk
        en_id2words[i+4] = s[0]
        en_ordered_words.append(s[0])

In [ ]:
#Reference Lab4 HW2
#Over 200000 loaded words, only 10 has wrong dimensions, simply discarded
words_to_load = 200000
datadir = os.getcwd()
with open(datadir + '/data/wiki.vi.vec') as f:
    loaded_zh_embeddings = np.zeros(((words_to_load+4),wiki_size))
    zh_word2id = {}
    zh_id2words = {}
    
    zh_id2words[PAD_TOKEN] = '<PAD>'
    zh_id2words[SOS_TOKEN] = '<SOS>'
    zh_id2words[EOS_TOKEN] = '<EOS>'
    zh_id2words[UNK_TOKEN] = '<UNK>'
    
    zh_word2id['<PAD>'] = PAD_TOKEN
    zh_word2id['<SOS>'] = SOS_TOKEN
    zh_word2id['<EOS>'] = EOS_TOKEN
    zh_word2id['<UNK>'] = UNK_TOKEN
    
    zh_ordered_words= []
    zh_ordered_words.append('<PAD>')
    zh_ordered_words.append('<SOS>')
    zh_ordered_words.append('<EOS>')
    zh_ordered_words.append('<UNK>')
    
    for i, line in enumerate(f):
        #print(i)
        if i >= words_to_load:
            break;
        if i == 0: #Ignore the first line
            continue;
        s = line.split()
        if len(s) != 301:
            print('Wrong dimension, skip')
            continue;
        loaded_zh_embeddings[i+4,:] = np.asarray(s[1:])
        zh_word2id[s[0]] = i+4 #for extra pad and unk 
        zh_id2words[i+4] = s[0]
        zh_ordered_words.append(s[0])

In [ ]:
#Reference Lab4 HW2
#Over 200000 loaded words, only 10 has wrong dimensions, simply discarded
words_to_load = 200000
datadir = os.getcwd()
with open(datadir + '/data/wiki.zh.vec') as f:
    loaded_zh_embeddings = np.zeros(((words_to_load+4),wiki_size))
    zh_word2id = {}
    zh_id2words = {}
    
    zh_id2words[PAD_TOKEN] = '<PAD>'
    zh_id2words[SOS_TOKEN] = '<SOS>'
    zh_id2words[EOS_TOKEN] = '<EOS>'
    zh_id2words[UNK_TOKEN] = '<UNK>'
    
    zh_word2id['<PAD>'] = PAD_TOKEN
    zh_word2id['<SOS>'] = SOS_TOKEN
    zh_word2id['<EOS>'] = EOS_TOKEN
    zh_word2id['<UNK>'] = UNK_TOKEN
    
    zh_ordered_words= []
    zh_ordered_words.append('<PAD>')
    zh_ordered_words.append('<SOS>')
    zh_ordered_words.append('<EOS>')
    zh_ordered_words.append('<UNK>')
    
    for i, line in enumerate(f):
        #print(i)
        if i >= words_to_load:
            break;
        if i == 0: #Ignore the first line
            continue;
        s = line.split()
        if len(s) != 301:
            print('Wrong dimension, skip')
            continue;
        loaded_zh_embeddings[i+4,:] = np.asarray(s[1:])
        zh_word2id[s[0]] = i+4 #for extra pad and unk 
        zh_id2words[i+4] = s[0]
        zh_ordered_words.append(s[0])

In [ ]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
BATCH_SIZE = 3
CUDA = False

class Lang:
    def __init__(self, name, emb_word2id, emb_id2word, emb_ordered_words):
        self.name = name
        self.word2index = emb_word2id
        self.word2count = {}
        self.index2word = emb_id2word #Dict
        self.n_words = 4  # Count SOS and EOS +(batch: pad and unk)

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2count:
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

#Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    #This line is commented out since it will not properly deal with Chinese Letters
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

#reference: LAB4 hw2
def indexesFromSentences(lang1, lang2, pairs):
    id_list1 = []
    id_list2 = []
    for i in range(len(pairs)):
        sentence1 = pairs[i][0]
        sentence2 = pairs[i][1]
        
        sentence1 = sentence1.replace('quot','')
        sentence1 = sentence1.replace('apos', '')
        sentence2 = sentence2.replace('quot','')
        sentence2 = sentence2.replace('apos', '')
        #If either sentence is empty, then remove the pair
        if sentence1 == '' or sentence2 == '':
            continue;
        
        id_sentence1 = [lang1.word2index[word] if word in lang1.word2index else UNK_TOKEN 
                        for word in sentence1.split()] + [EOS_TOKEN]
        id_list1.append(id_sentence1)
        id_sentence2 = [lang2.word2index[word] if word in lang2.word2index else UNK_TOKEN 
                        for word in sentence2.split()] + [EOS_TOKEN]
        id_list2.append(id_sentence2)
        
   
        
    return id_list1,id_list2


In [ ]:
pkl.dump(loaded_zh_embeddings, open('./data/zh_embeddings.p', 'wb'))
pkl.dump(loaded_en_embeddings, open('./data/en_embeddings.p', 'wb'))
pkl.dump(loaded_vi_embeddings, open('./data/vi_embeddings.p', 'wb'))

In [ ]:
def readLangs(lang1, lang2, category, reverse = False):#category = ['train', 'dev','test]
    print('Reading lines:')
    lines1 = open('data/iwslt-' + lang1.name +'-en/' + category +'.tok.'+ lang1.name, encoding = 'utf-8').\
    read().strip().split('\n')
    data1 = [normalizeString(l) for l in lines1]
    #data1 = list(filter(None, data1)) # fastest

    lines2 = open('data/iwslt-' + lang1.name +'-en/' + category + '.tok.' + lang2.name, encoding = 'utf-8').\
    read().strip().split('\n')
    data2 = [normalizeString(l) for l in lines2]
    #Given that data2 is english hence we further normalize
    data2 = [re.sub(r"[^a-zA-Z.!?]+", r" ", data) for data in data2]
    #data2 = list(filter(None, data2)) # fastest

    return data1, data2

In [ ]:
#Data Preparation for CHN to ENG
def prepareData(lang1, lang2, category, reverse = False):
    data1, data2 = readLangs(lang1, lang2, category, reverse)#Read data returns list of sentences
    pairs = [[data1[i], data2[i]] for i in range(len(data1))]
    print('Read %s sentence pairs' % len(pairs))
    #Count the words
    print('Counting words')
    for i in range(len(pairs)):
        lang1.addSentence(data1[i])
        lang2.addSentence(data2[i])

    print('Counted Words')
    print(lang1.name, lang1.n_words)
    print(lang2.name, lang2.n_words)

    return pairs, data1, data2



In [ ]:
#Create language object
input_zh = Lang('zh', zh_word2id, zh_id2words, zh_ordered_words)
output_zh_en = Lang('en', en_word2id, en_id2words, en_ordered_words)
input_vi = Lang('vi', vi_word2id, vi_id2words, vi_ordered_words)
output_vi_en = Lang('en', en_word2id, en_id2words, en_ordered_words)

In [ ]:
val_zh_pairs[0]

In [ ]:
#Create the string pairs and the string lists
train_zh_pairs, zh_train, zh_en_train = prepareData(input_zh, output_zh_en, 'train')
val_zh_pairs, zh_val, zh_en_val = prepareData(input_zh, output_zh_en, 'dev')
test_zh_pairs, zh_test, zh_en_test = prepareData(input_zh, output_zh_en, 'test')

train_vi_pairs, vi_train, vi_en_train = prepareData(input_vi, output_vi_en, 'train')
val_vi_pairs, vi_val, vi_en_val = prepareData(input_vi, output_vi_en, 'dev')
test_vi_pairs, vi_test, vi_en_test = prepareData(input_vi, output_vi_en, 'test')

In [ ]:
zh_idx_train, zh_en_idx_train = indexesFromSentences(input_zh, output_zh_en, train_zh_pairs)
zh_idx_val, zh_en_idx_val = indexesFromSentences(input_zh, output_zh_en, val_zh_pairs)
zh_idx_test, zh_en_idx_test = indexesFromSentences(input_zh, output_zh_en, test_zh_pairs)

vi_idx_train, vi_en_idx_train = indexesFromSentences(input_vi, output_vi_en, train_vi_pairs)
vi_idx_val, vi_en_idx_val = indexesFromSentences(input_vi, output_vi_en, val_vi_pairs)
vi_idx_test, vi_en_idx_test = indexesFromSentences(input_vi, output_vi_en, test_vi_pairs)


In [ ]:
zh_train_pairs = [[zh_idx_train[i], zh_en_idx_train[i]] for i in range(len(zh_idx_train))]
zh_val_pairs = [[zh_idx_val[i], zh_en_idx_val[i]] for i in range(len(zh_idx_val))]
zh_test_pairs= [[zh_idx_test[i], zh_en_idx_test[i]] for i in range(len(zh_idx_test))]
vi_train_pairs = [[vi_idx_train[i], vi_en_idx_train[i]] for i in range(len(vi_idx_train))]
vi_val_pairs = [[vi_idx_val[i], vi_en_idx_val[i]] for i in range(len(vi_idx_val))]
vi_test_pairs = [[vi_idx_test[i], vi_en_idx_test[i]] for i in range(len(vi_idx_test))]


In [ ]:
import pickle as pkl
pkl.dump(zh_train_pairs, open('./data/zh_train_pairs.p', 'wb'))
pkl.dump(zh_val_pairs, open('./data/zh_val_pairs.p', 'wb'))
pkl.dump(zh_test_pairs, open('./data/zh_test_pairs.p', 'wb'))

pkl.dump(vi_train_pairs, open('./data/vi_train_pairs.p', 'wb'))
pkl.dump(vi_val_pairs, open('./data/vi_val_pairs.p', 'wb'))
pkl.dump(vi_test_pairs, open('./data/vi_test_pairs.p', 'wb'))


In [ ]:
max_val_len = 0
second_len = 0
for pair in zh_val_pairs_cleaned:
    if max_val_len < len(pair[0]):
        second_len = max_val_len
        max_val_len = len(pair[0])

print(max_val_len, second_len)

In [ ]:
#For training data, we have max-len as 100: train: 212727/212922, val: 1260/1261, test: 1397/1397
zh_train_pairs_cleaned= []
zh_val_pairs_cleaned = []
zh_test_pairs_cleaned = []

for zh_list in zh_train_pairs:
    if len(zh_list[0])<=100 or len(zh_list[1]) <= 100:
        zh_train_pairs_cleaned.append(zh_list)
        
for zh_list in zh_val_pairs:
    if len(zh_list[0])<=100 or len(zh_list[1]) <= 100:
        zh_val_pairs_cleaned.append(zh_list)

for zh_list in zh_test_pairs:
    if len(zh_list[0])<=100 or len(zh_list[1]) <= 100:
        zh_test_pairs_cleaned.append(zh_list)


In [ ]:
#For training data, we have max-len as 100: train: 133038/133316, val: 1268/1268, test: 1553/1553
vi_train_pairs_cleaned= []
vi_val_pairs_cleaned = []
vi_test_pairs_cleaned = []

for vi_list in vi_train_pairs:
    if len(vi_list[0])<=100 or len(vi_list[1]) <= 100:
        vi_train_pairs_cleaned.append(vi_list)
        
for vi_list in vi_val_pairs:
    if len(vi_list[0])<=100 or len(vi_list[1]) <= 100:
        vi_val_pairs_cleaned.append(vi_list)

for vi_list in vi_test_pairs:
    if len(vi_list[0])<=100 or len(vi_list[1]) <= 100:
        vi_test_pairs_cleaned.append(vi_list)


In [ ]:
pkl.dump(zh_train_pairs_cleaned, open('./data/zh_train_pairs_cleaned.p', 'wb'))
pkl.dump(zh_val_pairs_cleaned, open('./data/zh_val_pairs_cleaned.p', 'wb'))
pkl.dump(zh_test_pairs_cleaned, open('./data/zh_test_pairs_cleaned.p', 'wb'))

pkl.dump(vi_train_pairs_cleaned, open('./data/vi_train_pairs_cleaned.p', 'wb'))
pkl.dump(vi_val_pairs_cleaned, open('./data/vi_val_pairs_cleaned.p', 'wb'))
pkl.dump(vi_test_pairs_cleaned, open('./data/vi_test_pairs_cleaned.p', 'wb'))


In [ ]:
class NMTDataset(Dataset):
    def __init__(self, pairs):#Needs the index pairs
        self.pairs = pairs
#         self.input_lang = input_lang
#         self.output_lang = output_lang
        self.input_seqs = [pairs[i][0] for i in range(len(self.pairs))]
        self.output_seqs = [pairs[i][1] for i in range(len(self.pairs))]

    def __len__(self):
        return len(self.pairs)#Returning number of pairs
    
    def __getitem__(self, index):
        input_seq = self.input_seqs[index]
        output_seq = self.output_seqs[index]
        return [input_seq, len(input_seq), output_seq, len(output_seq)]
    
def vocab_collate_func(batch):
    #Reference: lab8_3_mri
    def _pad_sequences(seqs):
        lens = [len(seq) for seq in seqs]
        padded_seqs = torch.zeros(len(seqs), max(lens)).long()
        for i, seq in enumerate(seqs):
            end = lens[i]
            padded_seqs[i, :end] = torch.LongTensor(seq[:end])
        return padded_seqs, lens
    
    batch_input_seqs = [datum[0] for datum in batch]
    batch_output_seqs = [datum[2] for datum in batch]
    #batch_input_length = [datum[1] for datum in batch]
    #batch_output_length = [datum[3] for datum in batch]

    sorted_pairs = sorted(zip(batch_input_seqs, batch_output_seqs), key=lambda x: len(x[0]), reverse = True)
    in_seq_sorted, out_seq_sorted = zip(*sorted_pairs)
    
    padded_input,input_lens = _pad_sequences(in_seq_sorted)
    padded_output,output_lens = _pad_sequences(out_seq_sorted)
    
    input_list = torch.from_numpy(np.array(padded_input))
    input_length = torch.LongTensor(input_lens)
    output_list = torch.from_numpy(np.array(padded_output))
    output_length = torch.LongTensor(output_lens)
    
    if CUDA:
        input_list = input_list.cuda()
        output_list = output_list.cuda()
        input_length = input_length.cuda()
        output_length = out_length.cuda()
            
    return [input_list, input_length, output_list, output_length]

In [ ]:
BATCH_SIZE = 32
'''
NMTDataset needs index pairs, need to call indexesFromPairs functions beforehand
The dataLoader is sorted according to length of the input_length, and padded to
max length of input and output list repectively
TODO: output_list is not sorted, hence need to sort (maybe) in the rnn sequence.
'''
# train_zh_dataset = NMTDataset(zh_train_pairs_cleaned, input_zh, output_zh_en)
# train_vi_dataset = NMTDataset(vi_train_pairs_cleaned, input_vi, output_vi_en)
# val_zh_dataset = NMTDataset(zh_val_pairs_cleaned, input_zh, output_zh_en)
# val_vi_dataset = NMTDataset(vi_val_pairs_cleaned, input_vi, output_vi_en)
# test_zh_dataset = NMTDataset(zh_test_pairs_cleaned, input_zh, output_zh_en)
# test_vi_dataset = NMTDataset(vi_test_pairs_cleaned, input_vi, output_vi_en)

train_zh_dataset = NMTDataset(zh_train_pairs_cleaned)
train_vi_dataset = NMTDataset(vi_train_pairs_cleaned)
val_zh_dataset = NMTDataset(zh_val_pairs_cleaned)
val_vi_dataset = NMTDataset(vi_val_pairs_cleaned)
test_zh_dataset = NMTDataset(zh_test_pairs_cleaned)
test_vi_dataset = NMTDataset(vi_test_pairs_cleaned)


train_zh_loader = torch.utils.data.DataLoader(dataset = train_zh_dataset, 
                                          batch_size = BATCH_SIZE,
                                          collate_fn = vocab_collate_func,
                                          shuffle = True)

train_vi_loader = torch.utils.data.DataLoader(dataset = train_vi_dataset, 
                                          batch_size = BATCH_SIZE,
                                          collate_fn = vocab_collate_func,
                                          shuffle = True)

#Will use batch size 1 for validation and test since the sentence will be translated one by one
val_zh_loader = torch.utils.data.DataLoader(dataset = val_zh_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
val_vi_loader = torch.utils.data.DataLoader(dataset = val_vi_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
test_zh_loader = torch.utils.data.DataLoader(dataset = test_zh_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
test_vi_loader = torch.utils.data.DataLoader(dataset = test_vi_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
#Input_batch in size Batch x maxLen


In [ ]:
#Here for the constant definition
MAX_SENTENCE_LENGTH = 10
hidden_size = 256
max_length = 10
BATCH_SIZE = 3
TEST_BATCH_SIZE = 3
CLIP = 50
TEACHER_RATIO = 0.5


In [ ]:
CUDA = False
loaded_zh_embeddings = torch.from_numpy(loaded_zh_embeddings).float()
loaded_vi_embeddings = torch.from_numpy(loaded_vi_embeddings).float()
loaded_en_embeddings = torch.from_numpy(loaded_en_embeddings).float()

if CUDA:
    loaded_zh_embeddings = loaded_zh_embeddings.cuda()
    loaded_vi_embeddings = loaded_vi_embeddings.cuda()
    loaded_en_embeddings = loaded_en_embeddings.cuda()


In [ ]:
def rnn_mask(length_list):
    max_length = length_list.max().item()
    masked_sentences = []
    longest_sentence = [1]*max_length
    for i in range(BATCH_SIZE):
        curr_length = length_list[i].item()
        masked_sentence = [1]*max_length
        masked_sentence[curr_length:] = [0] * (max_length - curr_length)
        masked_sentences.append(masked_sentence)
    if CUDA:
        masked_sentences = torch.from_numpy(np.array(masked_sentences)).cuda()
    else:
        masked_sentences = torch.from_numpy(np.array(masked_sentences))
    return masked_sentences
        
def rnn_mask_loss(decoder_outputs, output_list, output_length):
    '''
    decoder_outputs: 3d matrix containing all decoder output(B x output_lang vocab size)
                    while decoder_output is in size(max_len x vocab_size)
    output_list: Batch x max_len
    output_length: batch
    '''
    batch_size, max_len = output_list.size()
    decoder_outputs = decoder_outputs.view(-1, decoder_outputs.size(-1))#(bxmax_len) x V
    output_list = output_list.view(-1, 1)# (b x max_len) x 1 
    neg_loss = -torch.gather(decoder_outputs, 1, output_list)#(b x max_len) x 1
    neg_loss = neg_loss.view(batch_size, -1)# restore to b x max_len
    
    mask = rnn_mask(output_length)#b x max_len
    mask_loss = neg_loss * mask.float()
    
    loss = mask_loss.sum() / output_length.float().sum()
    return loss
        

In [ ]:
class PreBatchEncoderRNN(nn.Module):
    def __init__(self, emb, emb_size, hidden_size, n_layers=1, dropout=0.1):
        super(PreBatchEncoderRNN, self).__init__()
        
        #self.input_size = input_size
        self.hidden_size = hidden_size
        self.emb = emb
        self.emb_size = emb_size
        #self.batch_size = batch_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding.from_pretrained(self.emb, False, False)
        self.gru = nn.GRU(emb_size, hidden_size, n_layers, dropout=self.dropout, bidirectional = True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        '''
        input_seqs in size B x L sorted in decreasing order -> will transpose to fit in embedding dimension
        '''
        self.batch_size = input_seqs.size(0)
        #embedded size: max_len x B x H
        embedded = self.embedding(input_seqs.transpose(0,1))#input_seqs B x L -> transpose to L x B
        
        #Input length sorted by loader
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        #Outputs in shape L x B x 2H, hidden as the last state of the GRU
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        #outputs L x B x H
        #hidden size (2*n_layers) x B x H

        #outputs: seq_len x Batch x H
        return outputs, hidden
    
    def initHidden(self):
        #Due to bidrectional will have self.n_layers * 2
        return torch.zeros(self.n_layers *2, self.batch_size, self.hidden_size,device = device)#hidden size 2lays *B*H

In [ ]:
# RNN decoder with no attention used, batch implemented
# RNN decoder take one token at a time
class PreDecoderRNN(nn.Module):
    def __init__(self, emb, emb_size, hidden_size, output_size, n_layers=1, dropout_p = 0.1):
        super(PreDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.emb_size = emb_size
        #self.batch_size = batch_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = nn.Dropout(dropout_p)
        self.embedding = nn.Embedding.from_pretrained(emb, False, False)
        self.gru = nn.GRU(emb_size, hidden_size, n_layers, dropout = dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        self.batch_size = input.size(0)
        embedded = self.embedding(input).view(1, self.batch_size, -1)# 1 x B x E
        embedded = self.dropout(embedded)
        output = F.relu(embedded)
        output, hidden = self.gru(output, hidden)#output 1 x B x E, hidden n_layers x B x H
        out = self.out(output[0])
        out = self.softmax(out)
        #out size batch x output_lang_vocab_size
        #hidden n_layers x B x H
        return out, hidden
    
#     def initHidden(self):
#         return torch.zeros(self.n_layers, self.batch_size, self.hidden_size, device = device)

In [ ]:
#Reference and modified from 3-nmt
def no_attn_batch_train(input_list, input_length, output_list,output_length, 
                batch_encoder, batch_decoder, encoder_optimizer, decoder_optimizer, criterion):
    '''
    param: @attention is a Boolean variable indicating whether using attention
    '''
    batch_encoder.train()
    batch_decoder.train()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    max_output_length = output_length.max().item()

    
    loss = 0

    encoder_outputs, encoder_hidden = batch_encoder(input_list, input_length)

    #Initialize for decoding process
    curr_batch = input_list.size(0)#Take the current batch size
    decoder_input = torch.tensor([[SOS_TOKEN]]*curr_batch, device=device)
    
    decoder_hidden = encoder_hidden[:batch_decoder.n_layers]#Bidirectional summoned
    decoder_outputs = torch.zeros(max_output_length, curr_batch, batch_decoder.output_size)

    
    # Move new Variables to CUDA
    if CUDA:
        decoder_input = decoder_input.cuda()
        decoder_outputs = decoder_outputs.cuda()
    
    #use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    use_teacher_forcing = True
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(max_output_length):
            decoder_output, decoder_hidden = batch_decoder(
                decoder_input, decoder_hidden)
            decoder_outputs[di] = decoder_output
            decoder_input = output_list[:,di] # Teacher forcing
            loss += criterion(decoder_output, output_list[:,di])

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(max_output_length):
            decoder_output, decoder_hidden = batch_decoder(
                decoder_input, decoder_hidden)
            decoder_outputs[di] = decoder_input
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()# detach from history as input: size batch x 1 
            if ((decoder_output == EOS_TOKEN).sum().item()) == decoder_output.size(0):#If all are EOS tokens
                break;
            #loss += criterion(decoder_output, output_list[:,di])
            
    loss += rnn_mask_loss(decoder_outputs.transpose(0,1).contiguous(), output_list.contiguous(), output_length)
    #loss = criterion()      

    loss.backward()
    ec = torch.nn.utils.clip_grad_norm(batch_encoder.parameters(), CLIP)
    dc = torch.nn.utils.clip_grad_norm(batch_decoder.parameters(), CLIP)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()

In [ ]:
learning_rate = 0.01
encoder_optimizer = optim.SGD(pre_encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(pre_decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


pre_encoder = PreBatchEncoderRNN(loaded_zh_embeddings, emb_size, hidden_size, BATCH_SIZE)
pre_decoder = PreDecoderRNN(loaded_en_embeddings, emb_size, hidden_size, len(en_ordered_words), BATCH_SIZE)
encoder_outputs, encoder_hidden = pre_encoder(input_list, input_length)
decoder_input = torch.tensor([[SOS_TOKEN]]*BATCH_SIZE)
decoder_hidden = encoder_hidden[:no_attn_decoder.n_layers]
max_output_length = output_length.max().item()
decoder_outputs = torch.zeros(max_output_length, curr_batch, pre_decoder.output_size)
loss = 0
for di in range(max_output_length):
    #print(di)
    decoder_output, decoder_hidden = pre_decoder(
        decoder_input, decoder_hidden)
    decoder_outputs[di] = decoder_output
    decoder_input = output_list[:,di] # Teacher forcing
    loss += criterion(decoder_output, output_list[:,di])
print(loss.item()/max_output_length)

In [ ]:
pre_encoder = PreBatchEncoderRNN(loaded_zh_embeddings, emb_size, hidden_size, BATCH_SIZE).to(device)
pre_decoder = PreDecoderRNN(loaded_en_embeddings, emb_size, hidden_size, len(en_ordered_words), BATCH_SIZE)


learning_rate = 0.01
encoder_optimizer = optim.SGD(pre_encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(pre_decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

loss = no_attn_batch_train(input_list, input_length, output_list, output_length, 
                       pre_encoder, pre_decoder, encoder_optimizer, decoder_optimizer, 
                       criterion)

print(loss)



In [ ]:
#Reference lab8 1-nmt
def greedy_no_attn_evaluate(val_loader, encoder, decoder, en_id2words ):
    #Will generate sentences 1 by 1. 
    """
    Function that generate translation.
    First, feed the source sentence into the encoder and obtain the hidden states from encoder.
    Secondly, feed the hidden states into the decoder and unfold the outputs from the decoder.
    Lastly, for each outputs from the decoder, collect the corresponding words in the target language's vocabulary.
    And collect the attention for each output words.
    @param encoder: the encoder network
    @param decoder: the decoder network
    @param sentence: string, a sentence in source language to be translated
    @param max_length: the max # of words that the decoder can return
    @output decoded_words: a list of words in target language
    @output decoder_attentions: a list of vector, each of which sums up to 1.0
    """    
    # process input sentence
    decoded_words_all = []
    decoder_attentions_all = []
    
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        
        for i, (input_list, input_length, output_list, output_length) in enumerate(val_loader):
            #if i == 5:
            #    break
            #batch_size, max_len = output_list.size()
            print(input_list.size())
            
            # encode the source lanugage
            encoder_outputs, encoder_hidden = encoder(input_list, input_length)

            decoder_input = torch.tensor([[SOS_TOKEN]], device=device)  # SOS
            # decode the context vector
            decoder_hidden = encoder_hidden[:decoder.n_layers] # decoder starts from the last encoding sentence
            # output of this function
            decoded_words = []

            for di in range(MAX_LENGTH):
                # for each time step, the decoder network takes two inputs: previous outputs and the previous hidden states
                decoder_output, decoder_hidden = decoder(
                    decoder_input, decoder_hidden)

                top_score, topi = decoder_output.data.topk(1)
                decoder_attentions[di, :decoder_attention.size(-1)] = decoder_attention
                decoded_words.append(en_id2words[topi.item()])
                if topi.item() == EOS_TOKEN:
                    break
                else:
                    decoder_input = topi.squeeze().detach()
                    
            decoded_words_all.append(decoded_words)
            decoder_attentions_all.append(decoder_attentions[:di+1])

        return decoded_words_all, decoder_attentions_all

In [ ]:
#Translate the test and val lists back to english
def en_translate(index_list, en_id2words):
    translated_sentence_list = []
    for sentence in index_list:
        translated_sentence = []
        for index in sentence:
            translated_sentence.append(en_id2words[index])
        #translated_sentence.append('<EOS>')
        translated_sentence_list.append(translated_sentence)
    return translated_sentence_list
def post_process(decoded_words_all):
    cleaned_decoded_words_all = []
    
    for sentence in decoded_words_all:
        cleaned_sentence = []
        for word in sentence:
            if word == '<PAD>':
                continue
            else:
                cleaned_sentence.append(word)
        if cleaned_sentence[-1] != '<EOS>':
            cleaned_sentence.append(' <EOS>')
            
        cleaned_decoded_words_all.append(cleaned_sentence)
        
    return cleaned_decoded_words_all

In [ ]:
#Reference LAB8 1-nmt
def AttnTrainIters(train_loader, encoder, decoder, n_iters, print_every=100, plot_every=100, learning_rate=0.01,
                  val_translated_list):
    start = time.time()
    learning_curve_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
    for i in range(n_iters):
        losses = []
        for i, (input_list,input_length,output_list, output_length) in enumerate(train_loader):
            loss = attn_batch_train(input_list, input_length, output_list, output_length, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
        
            print_loss_total += loss
            plot_loss_total += loss
            learning_curve_losses.append(loss)
            
            if i > 0 and i % 500 == 0:
                decoded_val, decoder_attentions = greedy_attn_evaluate(val_loader, encoder, decoder, en_id2words)
                decoded_clean = post_process(decoded_val)

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
    torch.save(encoder.state_dict(), model_path + "encoder_rnn_atten.pth")
    torch.save(decoder.state_dict(), model_path + "decoder_rnn_atten.pth")

    showPlot(plot_losses)